<a href="https://colab.research.google.com/github/achreftel1/adc/blob/main/keras_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np

# Load the CIFAR-10 dataset
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Define the VGG model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(10, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define data augmentation
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# Define callbacks
checkpoint = ModelCheckpoint('vgg_model.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=int(len(x_train) / 32),
                    epochs=500,
                    validation_data=(x_test, y_test),
                    callbacks=[checkpoint, early_stopping, reduce_lr])

# Plot the error vs epoch graph
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

Epoch 1/500
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.3744 - loss: 1.8050
Epoch 1: val_loss improved from inf to 1.39571, saving model to vgg_model.keras
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 729s 466ms/step - accuracy: 0.3744 - loss: 1.8049 - val_accuracy: 0.5209 - val_loss: 1.3957 - learning_rate: 0.0010
Epoch 2/500
   1/1562 ━━━━━━━━━━━━━━━━━━━━ 14:26 555ms/step - accuracy: 0.5938 - loss: 1.1837
Epoch 2: val_loss improved from 1.39571 to 1.39541, saving model to vgg_model.keras
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 143s 91ms/step - accuracy: 0.5938 - loss: 1.1837 - val_accuracy: 0.5205 - val_loss: 1.3954 - learning_rate: 0.0010
Epoch 3/500
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.5113 - loss: 1.4292
Epoch 3: val_loss improved from 1.39541 to 1.31902, saving model to vgg_model.keras
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 777s 465ms/step - accuracy: 0.5113 - loss: 1.4292 - val_accuracy: 0.5430 - val_loss: 1.3190 - learning_rate: 0.0010
Epoch 4/500
   1/1562 ━━━━━━━━━━━━━━

KeyboardInterrupt: 